In [29]:
import pandas as pd
import numpy as np
import random

## Data Collection

In [30]:
def symmetricize(m, high_int=None):
    
    # if high_int not provided, make it equal to 10 times the max value:
    if high_int is None:
        high_int = round(10*m.max())
        
    m_bar = m.copy()
    np.fill_diagonal(m_bar, 0)
    u = np.matrix(np.ones(m.shape) * high_int)
    np.fill_diagonal(u, 0)
    m_symm_top = np.concatenate((u, np.transpose(m_bar)), axis=1)
    m_symm_bottom = np.concatenate((m_bar, u), axis=1)
    m_symm = np.concatenate((m_symm_top, m_symm_bottom), axis=0)
    
    return m_symm.astype(int) # Concorde requires integer weights


In [31]:
df = pd.read_csv('gb_cities.csv')

names = list(df['Place Name'].values)
durationMatrix = np.genfromtxt('CaliforniaDurationMatrix.csv', delimiter=',')
#durationMatrix = symmetricize(durationMatrix)

In [32]:
durationMatrix

array([[    0., 10892., 30364., ..., 23368., 25227., 19833.],
       [10901.,     0., 23625., ..., 16458., 18317., 13094.],
       [30326., 23541.,     0., ...,  8835.,  9441., 12260.],
       ...,
       [23395., 16444.,  9009., ...,     0.,  2790.,  7925.],
       [25275., 18324.,  9656., ...,  2859.,     0.,  9627.],
       [19854., 13069., 12341., ...,  8003.,  9637.,     0.]])

## Evolutionary Algorithm Functions

In [33]:
def initialize(populationSize):
    population = []
    for i in range(populationSize):
        randomSolution = names.copy()
        random.shuffle(randomSolution)
        population.append(randomSolution)
    return np.array(population)

In [34]:
def fitnessFunction(solution):
    totalDistance = 0
    for i in range(len(solution)-1):
        totalDistance += durationMatrix[names.index(solution[i])][names.index(solution[i+1])]

    return totalDistance

In [35]:
def evaluatePopulation(population):
    result = {}
    fitnessValues = []
    solutions = []

    for solution in population:
        fitnessValues.append(fitnessFunction(solution))
        solutions.append(solution)

    result['fitnessValues'] = fitnessValues
    centeredValues = [np.max(list(result["fitnessValues"]))-i for i in list(result["fitnessValues"])]
    result["weightedFitness"]  = [i/sum(centeredValues) for i in centeredValues]
    result["solution"] = np.array(solutions)

    return result


def RouletteWheel(population):
    
    evaluatedPopulation = evaluatePopulation(population)
    
    while (True):
        randomIndex = random.randint(0, len(population)-1)
        randomChance = evaluatedPopulation['weightedFitness'][randomIndex]

        if random.random() <= randomChance:
            pickedSolution = evaluatedPopulation['solution'][randomIndex]
            break
    
    return pickedSolution

In [36]:
def crossover(solution1, solution2):
    n = len(solution1)
    child = [np.nan for i in range(n)]
    num_els = np.ceil(n*(random.randint(10,90)/100))
    str_pnt = random.randint(0, n-2)
    end_pnt = n if int(str_pnt+num_els) > n else int(str_pnt+num_els)
    blockA = list(solution1[str_pnt:end_pnt])
    child[str_pnt:end_pnt] = blockA
    for i in range(n):
        if list(blockA).count(solution2[i]) == 0:
            for j in range(n):
                if pd.isna(child[j]):
                    child[j] = solution2[i]
                    break
    return child

In [37]:
def swap(solution, position1, position2):
    
    result = solution.copy()
    result[position1] = solution[position2]
    result[position2] = solution[position1] 
    return result

def swapMutation(solution):
    
    n = len(solution)
    pos_1 = random.randint(0,n-1)
    pos_2 = random.randint(0,n-1)
    
    return swap(solution, pos_1, pos_2)

# Main

In [38]:
generations = 200
populationSize = 20

In [39]:
# Create the initial population bag
population  = initialize(populationSize)
# Iterate over all generations
for generation in range(generations):
   # Calculate the fitness of elements in population bag
   populationFitness = evaluatePopulation(population)
   # Best individual so far
   bestFit = np.min(populationFitness["fitnessValues"])
   bestIndex = populationFitness["fitnessValues"].index(bestFit)
   bestSolution  = populationFitness["solution"][bestIndex]
   # Check if we have a new best
   if generation == 0:
      globalBestFit      = bestFit
      globalBestSolution = bestSolution
   else:
      if bestFit <= globalBestFit:
         globalBestFit      = bestFit
         globalBestSolution = bestSolution
   # Create the new population bag
   newPopulation = []
   for i in range(populationSize):
      # Pick 2 parents from the bag
      individual1 = RouletteWheel(population)
      individual2 = RouletteWheel(population)
      new_individual = individual1
      # Crossover the parents
      if random.random() <= 0.87:
         new_element = crossover(individual1, individual2)
      # Mutate the child
      if random.random() <= 0.7:
         new_element = swapMutation(new_element) 
      # Append the child to the bag
      newPopulation.append(new_element)
   # Set the new bag as the population bag
   population = np.array(newPopulation)

In [40]:
print(globalBestSolution)

['Derry' 'Lisburn' 'Belfast' 'St.Albans' 'Eastbourne' 'Oxford' 'York'
 'Perth' 'Cambridge' 'Portsmouth' 'Wells' 'Exeter' 'Solihull' 'Rugby'
 'Leicester' 'Basildon' 'Folkestone' 'London' 'Chesterfield' 'Bath'
 'Salisbury' 'Northampton' 'Canterbury' 'Chippenham' 'Derby' 'Ripon'
 'Truro' 'Bristol' 'Plymouth' 'Chelmsford' 'Brighton & Hove' 'Chichester'
 'Sutton Coldfield' 'Gloucester' 'Manchester' 'Salford' 'Southampton'
 'Lincoln' 'Gravesend' 'Swindon' 'Winchester' 'Wolverhampton' 'Lichfield'
 'Swansea' 'Worcester' 'Cardiff' 'Sunderland' 'Bedford' 'Birmingham'
 'Sheffield' 'Hereford' 'St. Davids' 'Carlisle' 'Leeds' 'Ely' 'Hastings'
 'Harlow' 'Lancaster' 'Coventry' 'Wakefield' 'Stoke-on-Trent' 'Chester'
 'Liverpool' 'Nottingham' 'Hull' 'Bradford' 'Peterborough'
 'Newcastle Upon Tyne' 'Edinburgh' 'Glasgow' 'Ayr' 'Preston' 'Durham'
 'Dundee' 'Elgin' 'Aberdeen' 'Inverness' 'Newry' 'Londonderry']
